In [1]:
import numpy as np

# 1

In [20]:
k = np.array([ [[0, 0],  [1, 1],  [0, 0]], 
               [[-2, 1], [0, -4], [2, 1]], 
               [[0, 0],  [1, 1],  [0, 0]]])

p = np.array([[[6, 3], [0, 1], [-4, -4], [0, 0], [1, -1]], 
              [[4, 3], [4, 0], [0, 3], [2, 4], [1, -1]], 
              [[3, 3], [-7, 7], [1, -2], [4, -2], [2, 2]], 
              [[-2, -5], [2, -2], [1, 2], [-4, 0], [2, -2]], 
              [[5, 2], [1, 1], [2, -1], [4, 1], [-1, 2]]])

In [21]:
nf = 2  # number of filters
rf = 3  # filter size
s = 2

In [41]:
out = np.zeros((2, 2))

In [42]:
for z in range(nf):
    
    h_range = int((p.shape[1] - rf) / s) + 1  # (W - F + 2P) / S
    for _h in range(h_range):
        w_range = int((p.shape[0] - rf) / s) + 1  # (W - F + 2P) / S
        for _w in range(w_range):
            # print(_h, _w)
            # print(np.sum((p[_h:_h+rf, _w:_w+rf, :]*k[:, :, :])))
            out[_h:, _w] = np.maximum((np.sum((p[_h:_h+rf, _w:_w+rf, :]*k[:, :, :]))), 0)

In [43]:
out

array([[ 0.,  0.],
       [ 0., 41.]])

# 2

In [25]:
from cvxopt import matrix, solvers

In [63]:
P = matrix([[2., 0, 4., 0.], [0., 2., 0., 4.], [4., 0., 8., 0.], [0., 4., 0., 8.]])
q = matrix([-1., -1., -1., -1.])
sol = solvers.qp(P, q)

In [65]:
print(sol['x'])

[ 1.13e+15]
[ 1.13e+15]
[-5.63e+14]
[-5.63e+14]



In [69]:
a = np.array([ 1.13e+15,1.13e+15,-5.63e+14,-5.63e+14])*np.array([1,-1,1,-1])

In [70]:
X = np.array([[1, 1], [-1, 1], [2, 2], [-2, 2]])

In [72]:
W = a[0]*X[0] + a[1]*X[1]+ a[2]*X[2] + a[3]*X[3]

In [73]:
W.T*X

array([[ 8.0e+12,  0.0e+00],
       [-8.0e+12,  0.0e+00],
       [ 1.6e+13,  0.0e+00],
       [-1.6e+13,  0.0e+00]])

# 3

In [34]:
N = 3
O = [0, 0, 1, 0, 1]
A = [[1/3, 1/3, 1/3], 
     [1/3, 1/3, 1/3], 
     [1/3, 1/3, 1/3]]
pi = [1/3, 1/3, 1/3]
B = [[4/6, 2/6], [2/6, 4/6], [3/6, 3/6]]
symbols = [0, 1]

def forward(N, O, pi, A, B):
    fwd = [{}]     
    for i in range(N):
        fwd[0][i] = pi[i] * B[i][O[0]]
    for t in range(1, len(O)):
        fwd.append({})     
        for j in range(N):
            fwd[t][j] = sum((fwd[t-1][i] *
                             A[i][j] * 
                             B[j][O[t]]) for i in range(N))
            
    prob = sum((fwd[len(O) - 1][i]) for i in range(N))
    return prob, fwd

def backward(N, O, A, B):
    T = len(O)
    bwk = [{} for t in range(T)]
    for i in range(N):
        bwk[T-1][i] = 1 
    for t in reversed(range(T-1)):
        for i in range(N):
            bwk[t][i] = sum((bwk[t+1][j] * A[i][j] * 
                             B[j][O[t+1]]) for j in range(N))
    prob = sum((pi[i]* B[i][O[0]] * 
                bwk[0][i]) for i in range(N))
    return prob, bwk

def train(N, O, pi, A, B, symbols):
    T = len(O)
    gamma = [{} for t in range(T)]
    zi = [{} for t in range(T - 1)] 
    p_o, fwd = forward(N, O, pi, A, B)
    _, bwk = backward(N, O, A, B)
    
    for t in range(T):
        for i in range(N):
            gamma[t][i] = (fwd[t][i] * bwk[t][i]) / p_o
            if t == 0:
                pi[i] = gamma[t][i]
            if t == T - 1:
                continue
            zi[t][i] = {}
            for j in range(N):
                zi[t][i][j] = (fwd[t][i] * A[i][j] * 
                B[j][O[t + 1]] * bwk[t + 1][j] / p_o)
    

    for i in range(N):
        for j in range(N):
            val = sum([zi[t][i][j] for t in range(T - 1)]) 
            val /= sum([gamma[t][i] for t in range(T - 1)])
            A[i][j] = val
            
    for i in range(N):
        for k in symbols:
            val = 0.0
            for t in range(T):
                if O[t] == k:
                    val += gamma[t][i]                 
            val /= sum([gamma[t][i] for t in range(T)])
            B[i][k] = val
    return pi, A, B

for iter_ in range(100):
    pi, A, B = train(N, O, pi, A, B, symbols)
    
print('π:', pi)
print('A:', A)
print('B:', B)

π: [1.0, 0.0, 0.0]
A: [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]]
B: [[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]]
